In [12]:
import numpy as np
import matplotlib.pyplot as plt
from pyblock2.driver.core import DMRGDriver, SymmetryTypes, MPOAlgorithmTypes, Block2Wrapper

# Tutorial Code for SZ mode

In [26]:
## Initial Script
L = 8
N = 8
TWOSZ = 0

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
driver.initialize_system(n_sites=L, n_elec=N, spin=TWOSZ)

In [27]:
## Build Hamiltonian
t = 1
U = 2

b = driver.expr_builder()

# hopping term
b.add_term("cd", np.array([[[i, i + 1], [i + 1, i]] for i in range(L - 1)]).flatten(), -t)
b.add_term("CD", np.array([[[i, i + 1], [i + 1, i]] for i in range(L - 1)]).flatten(), -t)

# onsite term
b.add_term("cdCD", np.array([[i, ] * 4 for i in range(L)]).flatten(), U)

mpo = driver.get_mpo(b.finalize(), iprint=2)


Build MPO | Nsites =     8 | Nterms =         36 | Algorithm = FastBIP | Cutoff = 1.00e-14
 Site =     0 /     8 .. Mmpo =     6 DW = 0.00e+00 NNZ =        6 SPT = 0.0000 Tmvc = 0.000 T = 0.002
 Site =     1 /     8 .. Mmpo =     6 DW = 0.00e+00 NNZ =       11 SPT = 0.6944 Tmvc = 0.000 T = 0.001
 Site =     2 /     8 .. Mmpo =     6 DW = 0.00e+00 NNZ =       11 SPT = 0.6944 Tmvc = 0.000 T = 0.002
 Site =     3 /     8 .. Mmpo =     6 DW = 0.00e+00 NNZ =       11 SPT = 0.6944 Tmvc = 0.000 T = 0.001
 Site =     4 /     8 .. Mmpo =     6 DW = 0.00e+00 NNZ =       11 SPT = 0.6944 Tmvc = 0.000 T = 0.001
 Site =     5 /     8 .. Mmpo =     6 DW = 0.00e+00 NNZ =       11 SPT = 0.6944 Tmvc = 0.000 T = 0.002
 Site =     6 /     8 .. Mmpo =     6 DW = 0.00e+00 NNZ =       11 SPT = 0.6944 Tmvc = 0.000 T = 0.002
 Site =     7 /     8 .. Mmpo =     1 DW = 0.00e+00 NNZ =        6 SPT = 0.0000 Tmvc = 0.000 T = 0.002
Ttotal =      0.014 Tmvc-total = 0.000 MPO bond dimension =     6 MaxDW = 0.00e+00
N

In [28]:
## Run DMRG
def run_dmrg(driver, mpo):
    ket = driver.get_random_mps(tag="KET", bond_dim=250, nroots=1)
    bond_dims = [250] * 4 + [500] * 4
    noises = [1e-4] * 4 + [1e-5] * 4 + [0]
    thrds = [1e-10] * 8
    return driver.dmrg(mpo, ket, n_sweeps=20, bond_dims=bond_dims, noises=noises,
        thrds=thrds, cutoff=0, iprint=1)

energies = run_dmrg(driver, mpo)
# print('DMRG energy = %20.15f' % energies)


Sweep =    0 | Direction =  forward | Bond dimension =  250 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.297 | E =     -49.8050731574 | DW = 3.68512e-16

Sweep =    1 | Direction = backward | Bond dimension =  250 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.453 | E =     -49.8050731574 | DE = 1.07e-13 | DW = 2.84294e-16

Sweep =    2 | Direction =  forward | Bond dimension =  250 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.577 | E =     -49.8050731574 | DE = -4.26e-14 | DW = 3.82309e-16

Sweep =    3 | Direction = backward | Bond dimension =  250 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.703 | E =     -49.8050731574 | DE = 2.84e-14 | DW = 2.93072e-16

Sweep =    4 | Direction =  forward | Bond dimension =  500 | Noise =  1.00e-05 | Dav threshold =  1.00e-10
Time elapsed =      1.058 | E =     -49.8050731574 | DE = -1.42e-14 | DW = 1.22851e-26

Sweep =    5 | Direction = backward | 

In [8]:
L = 8
U = 2
N_ELEC = 8
TWO_SZ = 0

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SAny, n_threads=4)

# quantum number wrapper (U1 / n_elec, U1 / 2*Sz)
driver.set_symmetry_groups("U1Fermi", "U1Fermi")
Q = driver.bw.SX

# [Part A] Set states and matrix representation of operators in local Hilbert space
site_basis, site_ops = [], []

for k in range(L):
    basis = [(Q(0, 0), 1), (Q(1, 1), 1), (Q(1, -1), 1), (Q(2, 0), 1)] # [0ab2]
    ops = {
        "": np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]),   # identity
        "c": np.array([[0, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 1, 0]]),  # alpha+
        "d": np.array([[0, 1, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]]),  # alpha
        "C": np.array([[0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [0, -1, 0, 0]]), # beta+
        "D": np.array([[0, 0, 1, 0], [0, 0, 0, -1], [0, 0, 0, 0], [0, 0, 0, 0]]), # beta
    }
    site_basis.append(basis)
    site_ops.append(ops)

In [9]:
print(site_basis)

[[(< U1F=0 U1F=0 >, 1), (< U1F=1 U1F=1 >, 1), (< U1F=1 U1F=-1 >, 1), (< U1F=2 U1F=0 >, 1)], [(< U1F=0 U1F=0 >, 1), (< U1F=1 U1F=1 >, 1), (< U1F=1 U1F=-1 >, 1), (< U1F=2 U1F=0 >, 1)], [(< U1F=0 U1F=0 >, 1), (< U1F=1 U1F=1 >, 1), (< U1F=1 U1F=-1 >, 1), (< U1F=2 U1F=0 >, 1)], [(< U1F=0 U1F=0 >, 1), (< U1F=1 U1F=1 >, 1), (< U1F=1 U1F=-1 >, 1), (< U1F=2 U1F=0 >, 1)], [(< U1F=0 U1F=0 >, 1), (< U1F=1 U1F=1 >, 1), (< U1F=1 U1F=-1 >, 1), (< U1F=2 U1F=0 >, 1)], [(< U1F=0 U1F=0 >, 1), (< U1F=1 U1F=1 >, 1), (< U1F=1 U1F=-1 >, 1), (< U1F=2 U1F=0 >, 1)], [(< U1F=0 U1F=0 >, 1), (< U1F=1 U1F=1 >, 1), (< U1F=1 U1F=-1 >, 1), (< U1F=2 U1F=0 >, 1)], [(< U1F=0 U1F=0 >, 1), (< U1F=1 U1F=1 >, 1), (< U1F=1 U1F=-1 >, 1), (< U1F=2 U1F=0 >, 1)]]


In [10]:
# [Part B] Set Hamiltonian terms
driver.initialize_system(n_sites=L, vacuum=Q(0, 0), target=Q(N_ELEC, TWO_SZ), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

b.add_term("cd", np.array([[i, i + 1, i + 1, i] for i in range(L - 1)]).ravel(), -1)
b.add_term("CD", np.array([[i, i + 1, i + 1, i] for i in range(L - 1)]).ravel(), -1)
b.add_term("cdCD", np.array([i for i in range(L) for _ in range(4)]), U)

In [14]:
# [Part C] Perform DMRG
mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops="cdCD"), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=250, nroots=1)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[250] * 4 + [500] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=1)
# print("DMRG energy = %20.15f" % energy)


Sweep =    0 | Direction =  forward | Bond dimension =  250 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.297 | E =      -6.2256341447 | DW = 3.26754e-16

Sweep =    1 | Direction = backward | Bond dimension =  250 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.631 | E =      -6.2256341447 | DE = -1.78e-15 | DW = 4.92271e-16

Sweep =    2 | Direction =  forward | Bond dimension =  250 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.907 | E =      -6.2256341447 | DE = -1.78e-14 | DW = 3.36213e-16

Sweep =    3 | Direction = backward | Bond dimension =  250 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      1.081 | E =      -6.2256341447 | DE = 6.22e-15 | DW = 4.16096e-16

Sweep =    4 | Direction =  forward | Bond dimension =  500 | Noise =  1.00e-05 | Dav threshold =  1.00e-10
Time elapsed =      1.655 | E =      -6.2256341447 | DE = 8.88e-16 | DW = 1.55623e-19

Sweep =    5 | Direction = backward | 

# My Code for Bose Hubbard Model

In [5]:
## Initial Script
## Testing with Block2Wrapper.set_symmetry_groups("U1")
L = 9
N = 9
TWOSZ = 0

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SAny, n_threads=4)
driver = Block2Wrapper.set_symmetry_groups("U1")
driver.initialize_system(n_sites=L, n_elec=N, spin=TWOSZ)

AttributeError: 'str' object has no attribute 'SXT'

In [31]:
L =4
np.array([[[i, i + 1], [i + 1, i]] for i in range(L - 1)]).flatten()

array([0, 1, 1, 0, 1, 2, 2, 1, 2, 3, 3, 2])

In [32]:
np.array([[i, ] * 4 for i in range(L)])

array([[0, 0, 0, 0],
       [1, 1, 1, 1],
       [2, 2, 2, 2],
       [3, 3, 3, 3]])

In [6]:
## Initial Script
## Testing with CPX
L = 9
N = 9
TWOSZ = 0

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.CPX, n_threads=4)
driver.initialize_system(n_sites=L, n_elec=N, spin=TWOSZ)

AttributeError: 'Block2Wrapper' object has no attribute 'SX'

In [7]:
## Initial Script
## Testing with SGB
L = 9
N = 9
TWOSZ = 0

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SGB, n_threads=4)
driver.initialize_system(n_sites=L, n_elec=N, spin=TWOSZ)

In [4]:
t = 1
U = 1

b = driver.expr_builder()

# hopping term
b.add_term("cd", np.array([[[i, i + 1], [i + 1, i]] for i in range(L - 1)]).flatten(), -t)

# onsite term
b.add_term("cd", np.array([[i, ] * 4 for i in range(L)]).flatten(), U)

mpo = driver.get_mpo(b.finalize(), iprint=2)


Build MPO | Nsites =     9 | Nterms =         25 | Algorithm = FastBIP | Cutoff = 1.00e-14
 Site =     0 /     9 .. Mmpo =     4 DW = 0.00e+00

IndexError: invalid unordered_map<K, T> key

In [17]:
### Using CustomHamiltonian Example

L = 2
U = 2
N_ELEC = 2
TWO_SZ = 0

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SAny, n_threads=4)

driver.set_symmetry_groups("U1")
Q = driver.bw.SX

In [19]:
# [Part A] Set states and matrix representation of operators in local Hilbert space
site_basis, site_ops = [], []

for k in range(L):
    basis = [(Q(2, 0), 1), (Q(1, 1), 1), (Q(2, 0), 1)] # [0ab2]
    ops = {
        "": np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]),   # identity
        "c": np.array([[0, 0, 0, 0], [1, 0, 0, 0], [0, np.sqrt(2), 0, 0], [0, 0, np.sqrt(3), 0]]),  # alpha+
        "d": np.array([[0, 1, 0, 0], [0, 0, np.sqrt(2), 0], [0, 0, 0, np.sqrt(3)], [0, 0, 0, 0]]),  # alpha
    }
    site_basis.append(basis)
    site_ops.append(ops)

AssertionError: 

In [20]:
# [Part B] Set Hamiltonian terms
driver.initialize_system(n_sites=L, vacuum=Q(0, 0), target=Q(N_ELEC, N_ELEC), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

b.add_term("cd", np.array([[i, i + 1, i + 1, i] for i in range(L - 1)]).ravel(), -1)
b.add_term("cd", np.array([i for i in range(L) for _ in range(4)]), U)

AssertionError: 

In [22]:
# [Part C] Perform DMRG
mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops="cdCD"), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=250, nroots=1)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[250] * 4 + [500] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=1)
# print("DMRG energy = %20.15f" % energy)